<a href="https://colab.research.google.com/github/DrumDong/Donggyu/blob/master/%EB%AA%A8%EB%91%90%EB%A5%BC%20%EC%9C%84%ED%95%9C%20%EB%94%A5%EB%9F%AC%EB%8B%9D/lab_11_2_MNIST99_with_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [31]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data',one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


# Conv layer 1

In [0]:
#input place holders
X = tf.placeholder(tf.float32,[None,784])
x_img = tf.reshape(X,[-1,28,28,1]) # img 28*28*1 (black/white), -1 : N개
Y = tf.placeholder(tf.float32,[None,10])

# L1 ImgIn shape= (?,28,28,1)
W1 = tf.Variable(tf.random_normal([3,3,1,32],stddev=0.01)) # 3*3의 이미지 색깔은 1, 32개의 filter 
# Conv -> (?,28,28,32)
# Pool -> (?,14,14,32)
L1 = tf.nn.conv2d(x_img,W1,strides=[1,1,1,1],padding='SAME')
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1, ksize=[1,2,2,1],
                    strides=[1,2,2,1],padding='SAME') # (28-2/2) +1 -> 14*14

14 * 14 * 32

##Conv layer2

In [0]:
# L2 ImgIn shape =(14 * 14 * 32)
W2 = tf.Variable(tf.random_normal([3,3,32,64],stddev=0.01))
# Conv -> (?,14,14,32)
# Pool -> (?,7,7,64)
L2 = tf.nn.conv2d(L1,W2,strides=[1,1,1,1],padding='SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
L2 = tf.reshape(L2,[-1,7*7*64])

## Fully Connected layer


In [0]:
#tf.reset_default_graph()

In [0]:
L2 = tf.reshape(L2,[-1,7*7*64])

#Final FC 7*7*64 inputs -> 10 outputs
W3 = tf.get_variable('W3',shape=[7*7*64,10],
                     initializer=tf.contrib.layers.xavier_initializer())
b = tf.Variable(tf.random_normal([10]))
hypothesis = tf.matmul(L2,W3)+b

#define cost/loss & optimizer
cost  = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=hypothesis,labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)

In [36]:
#initialize
sess = tf.Session()
sess.run(tf.global_variables_initializer())

traing_epochs = 15
batch_size  = 100

#train my model
print('Learning started. It takes some time.')
for epoch in range(traing_epochs):
  avg_cost = 0
  total_batch = int(mnist.train.num_examples / batch_size)
  for i in range(total_batch):
    batch_xs, batch_ys = mnist.train.next_batch(batch_size)
    c,_, = sess.run([cost,optimizer],
                    feed_dict = {X: batch_xs, Y: batch_ys})
    avg_cost += c /total_batch
  print('Epoch:','%04d' % (epoch+1),'cost=','{:.9f}'.format(avg_cost))

print('Learning Finished!')

#Test model and check accuracy
correct_prediction = tf.equal(tf.argmax(hypothesis,1),tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
print('Accuracy:',sess.run(accuracy,
                           feed_dict={X:mnist.test.images,Y:mnist.test.labels}))

Learning started. It takes some time.
Epoch: 0001 cost= 0.333305976
Epoch: 0002 cost= 0.091980636
Epoch: 0003 cost= 0.067091221
Epoch: 0004 cost= 0.054951407
Epoch: 0005 cost= 0.045296485
Epoch: 0006 cost= 0.039023035
Epoch: 0007 cost= 0.035513353
Epoch: 0008 cost= 0.029733252
Epoch: 0009 cost= 0.025892980
Epoch: 0010 cost= 0.023177381
Epoch: 0011 cost= 0.020018744
Epoch: 0012 cost= 0.018059235
Epoch: 0013 cost= 0.015579941
Epoch: 0014 cost= 0.013894943
Epoch: 0015 cost= 0.012023641
Learning Finished!
Accuracy: 0.9891
